
In epidemiology, a case fatality rate (CFR) — sometimes called case fatality risk — is the proportion of deaths from a certain disease compared to the total number of people diagnosed with the disease for a certain period of time. A CFR is conventionally expressed as a percentage and represents a measure of disease severity. CFRs are most often used for diseases with discrete, limited time courses, such as outbreaks of acute infections. A CFR can only be considered final when all the cases have been resolved (either died or recovered). The preliminary CFR, for example, during the course of an outbreak with a high daily increase and long resolution time would be substantially lower than the final CFR.

In [ ]:

# https://en.wikipedia.org/wiki/Case_fatality_rate

In [ ]:

command_str = '{sys.executable} -m pip install --upgrade statsmodels'.format(sys=sys)
print(command_str)
!{command_str}

In [1]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%pprint
%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

import scraping_utils
wikipedia = scraping_utils.wikipedia

from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
#pandemic_df = s.load_object('pandemic_df')
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])

dir()

Pretty printing has been turned OFF
D:\Documents\Repositories\notebooks\covid19\ipynb\Case Fatality Rates.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']


['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '__warningregistry__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'bs', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'example_iterrows', 'exit', 'filepath_regex', 'get_classifier', 'get_column_descriptions', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_max_rsquared_adj', 'get_module_version', 'get_notebook_path', 'get_page_tables', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'io', 'ipykernel', 'json', 'jupyter_config_dir', 'math', 'notebook_path', 'notebookapp', 'nx', 'os', 'pd', 'pickle', 'plt', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_wit

In [ ]:

!start %windir%\explorer.exe "{os.path.abspath(os.path.dirname(notebook_path))}"


---
# Scrape the Data

In [3]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_human_disease_case_fatality_rates'
tables_list = get_page_tables(tables_url)

[(0, (71, 5))]


In [12]:

CFRs_df = tables_list[0].copy()
CFRs_df.columns = ['Disease', 'Treatment', 'CFR', 'Notes', 'References']
columns_list = ['Disease', 'Treatment', 'CFR', 'Notes']
CFRs_df = CFRs_df[columns_list]
CFRs_df.CFR = CFRs_df.CFR.map(lambda x: re.sub('[^\d\.]+', '', str(x).split('–')[0]))
CFRs_df.CFR = pd.to_numeric(CFRs_df.CFR, errors='coerce')
s.store_objects(CFRs_df=CFRs_df)
CFRs_df.sample(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\CFRs_df.pickle


,Disease,Treatment,CFR,Notes
36,Dengue haemorrhagic fever (DHF),Untreated,26.0,Dengue haemorrhagic fever is also known as sev...
44,Intestinal capillariasis,Untreated,10.0,NaN
59,"Pertussis (whooping cough), children in develo...",Unvaccinated,1.0,For children 1–4 years old.
16,Cryptococcal meningitis,Co-infection with HIV,40.0,6 month mortality is >=60% with fluconazole-ba...
9,Smallpox Variola major – specifically the mali...,Untreated,95.0,The rate drops significantly to 10% with effec...
